## Synthetic DAPI stain generator

This notebook generates synthetic images of DAPI stains. The images could be used for training a variational auto-encoder (VAE) for testing purposes (e.g. to generate simulated test data of a FOV).

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt

import os

import numpy as np
import scipy.ndimage as ndi

from typing import Tuple

In [ ]:
# default point spread function kernel
DEFAULT_PSF_KERNEL = np.asarray(
    [[1.0, 0.5, 0.5],
     [0.5, 1.0, 0.5],
     [0.5, 0.5, 1.0]])

DEFAULT_PSF_KERNEL = DEFAULT_PSF_KERNEL / np.sum(DEFAULT_PSF_KERNEL)

class NucleusLatentSpace:
    """A parameter store for synthetic DAPI stains.
    
    Arguments:
        a_arr: 1d ndarray containing cosine coefficients
        b_arr: 1d ndarray containing sine coefficients
        theta_hnuc_arr: ndarray containing polar angles to place heterochromatin spots
        scale_hnuc: size of heterochromatin spots
        scale_boundary: size of the nucleus; must be in [0, 1]
        rel_radius_hnuc_arr: radial position of heterochromatin spots (relative to the nucleus boundary)
            must be in [0, 1]
        intensity_bg_min: minimum background intensity
        intensity_bg_max: maximum background intensity
        intensity_interior_min: minimum nucleus body intensity
        intensity_interior_max: maximum nucleus body intensity
        intensity_hnuc_min: minimum heterochromatin spot intensity
        intensity_hnuc_max: maximum heterochromatin spot intensity
        psf_kernel: 2d ndarray of point spread function 
    """
    def __init__(self,
                 a_arr,
                 b_arr,
                 theta_hnuc_arr,
                 scale_hnuc,
                 scale_boundary,
                 rel_radius_hnuc_arr,
                 intensity_bg_min = 0.0,
                 intensity_bg_max = 0.0,
                 intensity_interior_min = 0.1,
                 intensity_interior_max = 0.5,
                 intensity_hnuc_min = 0.4,
                 intensity_hnuc_max = 0.9,
                 psf_kernel = DEFAULT_PSF_KERNEL):
        assert len(a_arr) == len(b_arr)
        assert len(theta_hnuc_arr) == len(rel_radius_hnuc_arr)
        
        self.a_arr = a_arr
        self.b_arr = b_arr
        self.theta_hnuc_arr = theta_hnuc_arr
        self.scale_hnuc = scale_hnuc
        self.scale_boundary = scale_boundary
        self.rel_radius_hnuc_arr = rel_radius_hnuc_arr
        self.intensity_bg_min = intensity_bg_min
        self.intensity_bg_max = intensity_bg_max
        self.intensity_interior_min = intensity_interior_min
        self.intensity_interior_max = intensity_interior_max
        self.intensity_hnuc_min = intensity_hnuc_min
        self.intensity_hnuc_max = intensity_hnuc_max
        self.psf_kernel = psf_kernel
        
        # enforce the coefficient of cos(0) is 1.0
        self.a_arr[0] = 1.0
        

def generate_random_nucleus_params() -> NucleusLatentSpace:
    """Generates sensible random instances of `NucleusLatentSpace`."""
    scale_boundary = 0.5
    num_harmonics = 6
    harmonics_attenuation_factor = 0.2 / np.arange(1, num_harmonics + 1)
    a_arr = np.random.randn(num_harmonics) * harmonics_attenuation_factor
    b_arr = np.random.randn(num_harmonics) * harmonics_attenuation_factor
    
    scale_hnuc = 0.07
    num_hnuc = np.random.poisson(5.)
    theta_hnuc_arr = 2 * np.pi * np.random.rand(num_hnuc)
    rel_radius_hnuc_arr = np.random.rand(num_hnuc)
    
    return NucleusLatentSpace(a_arr, b_arr, theta_hnuc_arr, scale_hnuc, scale_boundary, rel_radius_hnuc_arr)


def get_boundary_radius(a_arr, b_arr, theta, scale_boundary) -> np.ndarray:
    """Returns the boundary radius of the nucleus.
    
    Arguments:
        a_arr: 1d ndarray containing cosine coefficients
        b_arr: 1d ndarray containing sine coefficients
        theta: ndarray containing polar angles for which the radius is to be calculated
        scale_boundary: a global scale factor for the boundary curve
    """
    cos_theta = np.zeros(theta.shape + (len(a_arr),))
    sin_theta = np.zeros(theta.shape + (len(a_arr),))
    for n in range(len(a_arr)):
        cos_theta[..., n] = np.cos(n * theta)
        sin_theta[..., n] = np.sin(n * theta)
    return scale_boundary * np.sum((a_arr * cos_theta + a_arr * sin_theta), axis=-1)


def get_uniform_random(min_val: float, max_val: float, shape: Tuple) -> np.ndarray:
    """Returns uniform random number in a given interval."""
    return min_val + (max_val - min_val) * np.random.rand(*shape)


def generate_dapi_stain(params: NucleusLatentSpace, width=128, height=128) -> np.ndarray:
    """Generates a synthetic DAPI stain for given parameters."""
    # generate grid
    x_vec = np.linspace(-1.0, 1.0, num=width)
    y_vec = np.linspace(-1.0, 1.0, num=height)
    x_mat, y_mat = np.meshgrid(x_vec, y_vec)
    
    # generate polar coordinates
    r_mat = np.sqrt(x_mat**2 + y_mat**2)
    t_mat = np.arctan2(y_mat, x_mat)
    
    boundary_radius_mat = get_boundary_radius(params.a_arr, params.b_arr, t_mat, params.scale_boundary)
    interior_mask = r_mat < boundary_radius_mat

    full_hnuc_mask = np.zeros_like(interior_mask)
    num_hnuc = len(params.theta_hnuc_arr)
    for n in range(num_hnuc):
        radius_boundary = get_boundary_radius(
            params.a_arr, params.b_arr, np.asarray([params.theta_hnuc_arr[n]]), params.scale_boundary)
        radius_hnuc = params.rel_radius_hnuc_arr[n] * (radius_boundary - params.scale_hnuc)
        x_hnuc = radius_hnuc * np.cos(params.theta_hnuc_arr[n])
        y_hnuc = radius_hnuc * np.sin(params.theta_hnuc_arr[n])
        single_hnuc_mask = np.sqrt((x_mat - x_hnuc)**2 + (y_mat - y_hnuc)**2) < params.scale_hnuc
        full_hnuc_mask += single_hnuc_mask

    shape = interior_mask.shape

    full_image = get_uniform_random(
        params.intensity_bg_min, params.intensity_bg_max, shape)
    full_image[interior_mask] = get_uniform_random(
        params.intensity_interior_min, params.intensity_interior_max, full_image[interior_mask].shape)
    full_image[full_hnuc_mask] = get_uniform_random(
        params.intensity_hnuc_min, params.intensity_hnuc_max, full_image[full_hnuc_mask].shape)

    # apply PSF
    final_image = ndi.convolve(full_image, params.psf_kernel)
    
    return final_image


def generate_batch(width, height, num_samples) -> np.ndarray:
    """Generates a batch of synthetic DAPI stains."""
    synth_dapi_stain_samples = np.zeros((num_samples, width, height))

    for n in range(num_samples):
        synth_dapi_stain_samples[n, ...] = generate_dapi_stain(
            generate_random_nucleus_params(), width, height)
    return synth_dapi_stain_samples


def show_images(imgs) -> None:
    plt.figure(figsize=(10, 2))
    for i, img in enumerate(imgs):
        plt.subplot(1, len(imgs), i + 1)
        plt.axis('off')
        plt.imshow(img, cmap='gray')

## Test

In [ ]:
show_images(generate_batch(64, 64, 5))

## Generate and Save

In [ ]:
np.save('./synth_dapi_stains/synth_dapi_stains_train_64_64_10k.npy',
        generate_batch(64, 64, 10_000))

In [ ]:
np.save('./synth_dapi_stains/synth_dapi_stains_test_64_64_1k.npy',
        generate_batch(64, 64, 1_000))